In [1]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_atom as E9a
import E9_fn.E9_cooltrap as E9ct
import E9_fn.polarizabilities_calculation as E9pol
# import E9_fn.datasets.transition_line_data as TLData
from E9_fn import util

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

c:\Users\ken92\Documents\Studies\E5\simulation\E9_simulations\.venv\Lib\site-packages\gftool\precision.py:16: UserWarning: No quad precision datatypes available!
Some functions might be less accurate.
  warnings.warn("No quad precision datatypes available!\n"


# Atom stuff

In [2]:
Rb_atom = E9a.Rb87_5_2S1o2_F2
mu_Rb = E9c.mu_B * Rb_atom.gF * 2   # 2 here is mF = 2; sign is V = mu * B
K_atom = E9a.K40_4_2S1o2_F9o2
mu_K = E9c.mu_B * K_atom.gF * 9/2

## Magnetic trap

In [3]:
B_grad_com = 100        # [G/cm] B field gradient after compressing
B_grad_decom = 20       # [G/cm] B field gradient after decompressing
delta_rf_final = -10e6  # [MHz] detuning of RF knife at the end of rf evaporation
Rb87_mF = 2             # mF value

V_grad_decom = (B_grad_decom / 1e4 * 1e2) * E9c.mu_B * E9a.Rb87_5_2S1o2_F2.gF * Rb87_mF # [J/m]
V_grad_decom_uK__mm = V_grad_decom / E9c.k_B / 1e-6 / 1e3
V_grad_com = (B_grad_com / 1e4 * 1e2) * E9c.mu_B * E9a.Rb87_5_2S1o2_F2.gF * Rb87_mF # [J/m]
V_grad_com_uK__mm = V_grad_com / E9c.k_B / 1e-6 / 1e3
V_rf_evap_uK = delta_rf_final / 20e9 / 1e-6
print("V_grad_decom = {:.4f} uK/mm".format(V_grad_decom_uK__mm))
print("V_rf_evap = {:.4f} uK".format(V_rf_evap_uK))
print("(semiclassical) decompressed trap size = {:.4f} mm".format(abs(V_rf_evap_uK) / V_grad_decom_uK__mm))

V_grad_decom = 134.3428 uK/mm
V_rf_evap = -500.0000 uK
(semiclassical) decompressed trap size = 3.7218 mm


## ODTs

In [4]:
pwr_ODT = 1     # [W] ODT power at atoms
w0_ODT = 50e-6  # ODT waist (not using the value I have in E9_constants)
fattening = 1.  # Additional fudge factor to account for being off focus, M2 > 1, etc.

I_ODT_max = util.I_from_power(pwr_ODT, w0_ODT * fattening)
V_ODT = E9ct.V0_from_I(E9c.gamma_Rb87_D2, (E9c.nu_Rb87_4_2P3o2 + E9c.nu_Rb87_4_2P1o2)/2,
                      E9c.f_lw, I_ODT_max, E9a.Rb87_5_2S1o2_F2.gF, 2)
print("Fattage of the beam = {} (beam waist = {:.2e})".format(fattening, w0_ODT * fattening))
print("V_ODT = {:.4f} uK".format(V_ODT / E9c.k_B / 1e-6))
print("V_ODT = {:.4f} kHz".format(V_ODT / E9c.hnobar / 1000))

Fattage of the beam = 1.0 (beam waist = 5.00e-05)
V_ODT = -35.0160 uK
V_ODT = -729.6146 kHz


## DMD

In [8]:
# input parameters
lambda_DMD = 638e-9     # wavelength used for DMD potential
pwr_DMD = 0.5             # DMD power at atoms, in watts
DMD_FoV = 200e-6**2     # field of view (region illuminated) by the DMD

I_at_atom = pwr_DMD / DMD_FoV
f_DMD = E9c.c_light / lambda_DMD
V_DMD = E9ct.V0_from_I(E9c.gamma_40K_D2, (E9c.nu_K40_4_2P3o2 + E9c.nu_K40_4_2P1o2)/2,
                      f_DMD, I_at_atom, E9a.K40_4_2S1o2_F9o2.gF, 9/2)
print("V_DMD = {} kHz".format(V_DMD / E9c.hnobar / 1000))


V_DMD = 41.047169938025675 kHz


In [9]:
Rb_pol = E9pol.alpha_s_Rb_5S1o2(lambda_DMD)
K_pol = E9pol.alpha_s_K_4S1o2(lambda_DMD)

I_at_atom = pwr_DMD / DMD_FoV
V_uK_Rb_plug = E9pol.I2uK_from_pol(I_at_atom, Rb_pol)
V_uK_K_plug = E9pol.I2uK_from_pol(I_at_atom, K_pol)
V_kHz_Rb_plug = V_uK_Rb_plug * 1e-6 * E9c.k_B / E9c.hnobar / 1e3
V_kHz_K_plug = V_uK_K_plug * 1e-6 * E9c.k_B / E9c.hnobar / 1e3
print(f"(Anti-)trap depth of the DMD for Rb: {V_kHz_Rb_plug:.2f} kHz ({V_uK_Rb_plug:.2f} uK)")
print(f"(Anti-)trap depth of the DMD for K: {V_kHz_K_plug:.2f} kHz ({V_uK_K_plug:.2f} uK)")

INFO:root:K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_4D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_4D3o2 transition does not have f_ik data (not E1 allowed?)


(Anti-)trap depth of the DMD for Rb: 35.17 kHz (1.69 uK)
(Anti-)trap depth of the DMD for K: 36.97 kHz (1.77 uK)


In [10]:
w_MOT = 6e-3
I_MOT = 60e-3
gamma_Yb174 = 182.4e3 * 2 * np.pi
lambda_Yb174 = 555.8e-9
m_Yb174 = 174 * E9c.m_u
mu_Yb174 = 1.5 * E9c.mu_B
v_MOT_actual = 5

k_MOT = 2 * np.pi / lambda_Yb174
v_MOT_max = np.sqrt(2 * E9c.hbar * k_MOT * gamma_Yb174 * w_MOT / m_Yb174)
A_max = (E9c.hbar * k_MOT)**2 / (2 * m_Yb174) * gamma_Yb174 / (mu_Yb174 * v_MOT_actual)
print(v_MOT_max)
print("A_max = {} G/cm".format(A_max * 100))

7.532905935793225
A_max = 4.052495215875869 G/cm
